# Publish data in public folder

In [1]:
from pathlib import Path
import sys
import os
import shutil
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import gsw
from IPython.display import display,Image

home = str(Path.home())
processdir = Path(home,'MEOP_process')
path_MEOP_analyses = os.fspath(Path(home,'MEOP_analyses/'))
if not (path_MEOP_analyses in sys.path):
        sys.path.insert(0,path_MEOP_analyses)

import importlib
import meop
importlib.reload(meop)

def copy_file(file_name,src_dir,dst_dir):
    shutil.copyfile(Path(src_dir)/file_name,Path(dst_dir)/file_name)

In [2]:
# initialize arborescence
MEOP_version = 'MEOP-CTD_2020-08-29'
folder_public = Path.home() / 'public' / MEOP_version
folder_public.mkdir(parents=True, exist_ok=True)
copy_file('README_licenseODbl.txt',meop.processdir,folder_public)

In [3]:
# read list of profiles
list_data = meop.read_list_data_in_MEOP()
# select only profiles will at least one data point
list_data = list_data[(list_data.N_TEMP>0)|(list_data.N_PSAL>0)|(list_data.N_CHLA>0)]

# read list of deployments
list_depl = meop.read_list_deployment()
list_depl = list_depl[list_depl.PUBLIC==1]

# merge the two tables
list_profiles = list_data.reset_index().drop_duplicates()
list_profiles = list_depl.loc[:,['PI_CODE','COUNTRY']].merge(list_profiles,on='DEPLOYMENT_CODE',how='inner')
list_profiles['N_PROF'] = list_profiles.groupby('SMRU_PLATFORM_CODE').N_TEMP.transform(lambda x: len(x))

In [4]:
list_profiles

,DEPLOYMENT_CODE,PI_CODE,COUNTRY,SMRU_PLATFORM_CODE,CYCLE_NUMBER,JULD,LATITUDE,LONGITUDE,N_TEMP,N_PSAL,N_CHLA,N_PROF
0,wd04,HINDELL,AUSTRALIA,wd04-896-11,1,2011-03-25 11:40:00.000001,-68.402667,78.425164,28,0,0.0,572
1,wd04,HINDELL,AUSTRALIA,wd04-896-11,2,2011-03-25 14:00:00.000003,-68.406358,78.402009,16,0,0.0,572
2,wd04,HINDELL,AUSTRALIA,wd04-896-11,3,2011-03-25 17:40:00.000001,-68.408168,78.379959,9,0,0.0,572
3,wd04,HINDELL,AUSTRALIA,wd04-896-11,4,2011-03-25 20:29:59.999997,-68.411473,78.387576,18,0,0.0,572
4,wd04,HINDELL,AUSTRALIA,wd04-896-11,5,2011-03-26 02:20:00.000002,-68.406252,78.359768,28,0,0.0,572
...,...,...,...,...,...,...,...,...,...,...,...,...
602379,ct83,GOEBEL,USA,ct83-859-13,337,2014-06-19 20:29:59.999997,-62.324400,-60.801100,16,16,0.0,341
602380,ct83,GOEBEL,USA,ct83-859-13,338,2014-06-20 03:10:00.000004,-62.323200,-60.946100,16,16,0.0,341
602381,ct83,GOEBEL,USA,ct83-859-13,339,2014-06-20 12:10:00.000004,-62.289400,-60.876400,16,16,0.0,341
602382,ct83,GOEBEL,USA,ct83-859-13,340,2014-06-21 03:59:59.999997,-62.353100,-60.765100,8,8,0.0,341


In [5]:
# write list of profiles in a csv file
list_profiles.to_csv(MEOP_version+'_list_profiles.csv')
copy_file(MEOP_version+'_list_profiles.csv','.',folder_public)

In [6]:
# list public tags
list_tags = list_profiles.loc[:,['DEPLOYMENT_CODE','PI_CODE','COUNTRY','SMRU_PLATFORM_CODE']].drop_duplicates()
list_tags

,DEPLOYMENT_CODE,PI_CODE,COUNTRY,SMRU_PLATFORM_CODE
0,wd04,HINDELL,AUSTRALIA,wd04-896-11
572,wd04,HINDELL,AUSTRALIA,wd04-880-11
1213,wd04,HINDELL,AUSTRALIA,wd04-900-11
1530,wd04,HINDELL,AUSTRALIA,wd04-883-11
2105,wd04,HINDELL,AUSTRALIA,wd04-908-11
...,...,...,...,...
600302,ct57,GOEBEL,USA,ct57-L476-10
600885,ct57,GOEBEL,USA,ct57-L479-10
601497,ct57,GOEBEL,USA,ct57-W483-10
601831,ct57,GOEBEL,USA,ct57-L478-10


In [7]:
# for COUNTRY in list_tags.COUNTRY.unique():
#     print(COUNTRY)
#     folder_country = folder_public / COUNTRY
#     folder_country.mkdir(parents=True, exist_ok=True)
#     folder_data = folder_country / 'DATA'
#     folder_data.mkdir(parents=True, exist_ok=True)
#     list_tags_country = list_tags.loc[list_tags.COUNTRY==COUNTRY,['SMRU_PLATFORM_CODE']]
#     for tag in list_tags_country.values[:,0]:
#         ncfile_in = Path(meop.fname_prof(tag,qf='hr2'))
#         ncfile_out = folder_data / ncfile_in.name
#         if ncfile_in.exists() and (not ncfile_out.exists()):
#             shutil.copyfile(ncfile_in,ncfile_out)

In [8]:
%matplotlib auto
importlib.reload(meop)
for COUNTRY in list_tags.COUNTRY.unique():
    print(COUNTRY)
    folder_plots = folder_public / COUNTRY / 'PLOTS'
    folder_plots.mkdir(parents=True, exist_ok=True)
    list_tags_country = list_tags.loc[list_tags.COUNTRY==COUNTRY,['SMRU_PLATFORM_CODE']]
    for tag in list_tags_country.values[:,0]:
        print(tag)
        if meop.fname_prof(tag,qf='hr2').is_file():
            with meop.read_ncfile(tag,qf='hr2') as ds:
                ds = ds.assign_coords(pressure=("N_LEVELS", ds.PRES[0,:]))
                ds['SIG0_ADJUSTED'] = (('N_PROF','N_LEVELS'),gsw.sigma0(ds.PSAL_ADJUSTED,ds.TEMP_ADJUSTED))
                depl = meop.EXP_from_SMRU_CODE(tag)
                namefig = folder_plots / (tag+'_data_description.png')
                if not namefig.exists():
                    meop.plot_data_tags(ds,namefig=namefig)
                    plt.close()

Using matplotlib backend: agg
AUSTRALIA
wd04-896-11
wd04-880-11
wd04-900-11
wd04-883-11
wd04-908-11
wd04-897-11
wd04-899-11
wd04-881-11
wd04-838-11
wd04-884-11
wd04-909-11
wd04-906-11
wd04-910-11
wd04-907-11
wd04-898-11
wd04-911-11
wd04-836-11
awru1-B-06
awru1-C-06
awru1-A-06
ct149-928-18
ct149-924-18
ct149-930-18
ct149-923-18
ct149-922-18
ct149-929-18
ct149-926-18
ct149-925-18
ct149-927-18
ct2-9920-04
ct2-9919-04
ct2-9925-04
ct2-9916-04
ct36-E-09
ct36-D-09
ct36-R1-09
ct36-A-09
ct36-R4-09
ct36-C-09
ct36-B-09
ct36-F-09
ct38w-Elodie-08
ct38w-QueenEliz-08
ct38w-Denise-08
ct38w-Mathilde-08
ct47-I-09
ct47-G-09
ct47-C-09
ct47-E-09
ct47-F-09
ct47-A-09
ct47-B-09
ct47-D-09
ct6-10006-05
ct6-10018-05
ct6-9920-05
ct6-10010-05
ct6-10015-05
ct6-10016-05
ct6-10021-05
ct6-10011-05
ct6-10020-05
ct6-10017-05
ct75-839-11
ct75-020-11
ct75-030-11
ct75-Cy32-11
ct75-838-11
ft24-919-18
ft24-920-18
ft24-918-18
ft24-917-18
ft24-921-18
wd1-9497-06
wd1-10213-06
wd1-10216-06
wd1-10217-06
wd1-10165-06
wd1-10183-06


In [9]:
namefig = folder_plots / ('wd04-882-11'+'_data_description.png')
meop.show_png(namefig)

File not found: /home/jupyter-froqu/public/MEOP-CTD_2020-08-29/USA/PLOTS/wd04-882-11_data_description.png
